# Blended Queries in Elasticsearch

In [1]:
import os
import tika
tika.initVM()
from tika import parser  
import re
from datetime import date
import pandas as pd
import json
from datetime import datetime
import requests
import PyPDF2

from pathlib import Path

In [2]:
ES_URL ='Add your endpoint here'

In [5]:
from elasticsearch import Elasticsearch

es = Elasticsearch(
    'Add your Elastic Search endpoint here',
     headers={'Authorization': 'apikey  Add your API key here'},
    
)


In [6]:
info = es.info()
print(info)

{'name': 'instance-0000000001', 'cluster_name': '8f0e4b50fdb54cbdbaa9768dce87cce0', 'cluster_uuid': 'KNkOkY30RoGT2iZ-ewHtPA', 'version': {'number': '8.8.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'f8edfccba429b6477927a7c1ce1bc6729521305e', 'build_date': '2023-06-05T21:32:25.188464208Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [189]:
## Create a pipline for 144 questions

#!pip3 install openpyxl

In [7]:
import pandas as pd
# Read Excel file
df = pd.read_excel('1686845308-EVALUATION_GOLD_URLS.xlsx')
print(df)

                                              question  Category   \
0    How do I get ready for using Sourcing and Proc...  Indirect    
1    What features does SAP Sourcing and Procuremen...     Direct   
2    What integration scenarios are available for S...     Direct   
3    Can I connect my SAP S/4HANA Cloud with extern...  Indirect    
4                    What does Central Procurement do?     Direct   
..                                                 ...        ...   
139     How can I claim my travel expenses in Concur?     Deviant   
140                How can I claim my travel expenses?    Deviant   
141                How can I place an order in Ariba?     Deviant   
142                 What is our USP against Microsoft?    Deviant   
143               When is the birthday of my manager?     Deviant   

                                                answer  \
0    I do not have information regarding Sourcing a...   
1    I do not have information regarding SAP Sourci... 

In [8]:
df.columns

Index(['question', 'Category ', 'answer', 'golden_anwser', 'PROMPT',
       'wrong_urls', 'wrong_passages', 'evaluation', 'comments', 'passage_1',
       'passage_1_id', 'passage_2', 'passage_2_id', 'passage_3',
       'passage_3_id', 'discovery_passage_1', 'discovery_passage_1_id',
       'discovery_passage_2', 'discovery_passage_2_id', 'discovery_passage_3',
       'discovery_passage_3_id', 'proposed-gold-url-1', 'proposed-gold-url-2',
       'proposed-gold-url-3', 'gold-url-1', 'gold-url-2', 'gold-url-3',
       'reranker_passage_1', 'reranker_passage_1_id', 'reranker_passage_2',
       'reranker_passage_2_id', 'reranker_passage_3', 'reranker_passage_3_id',
       'elastic_search_passage_1', 'elastic_search_passage_1_id',
       'elastic_search_passage_2', 'elastic_search_passage_2_id',
       'elastic_search_passage_3', 'elastic_search_passage_3_id'],
      dtype='object')

In [9]:
df_sap = df[['question','Category ','proposed-gold-url-1', 'proposed-gold-url-2',
       'proposed-gold-url-3', 'gold-url-1', 'gold-url-2', 'gold-url-3']]

In [10]:
df_sap.columns

Index(['question', 'Category ', 'proposed-gold-url-1', 'proposed-gold-url-2',
       'proposed-gold-url-3', 'gold-url-1', 'gold-url-2', 'gold-url-3'],
      dtype='object')

In [11]:
df_sap.rename(columns = {'Category ':'Category'}, inplace = True)

/var/folders/lp/ydkpkqws5mv6ptd1fy7p5w_40000gn/T/ipykernel_30661/766358501.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sap.rename(columns = {'Category ':'Category'}, inplace = True)


In [12]:
df_sap.Category.unique()

array(['Indirect ', 'Direct', 'Ambiguous ', 'Aggregated', 'Comparative',
       'Deviant'], dtype=object)

In [13]:
### Collect the questions based on Category
df_sap['url1'] =df_sap['gold-url-1']
df_sap['url2'] =df_sap['gold-url-2']
df_sap['url3'] =df_sap['gold-url-3']

/var/folders/lp/ydkpkqws5mv6ptd1fy7p5w_40000gn/T/ipykernel_30661/1230006049.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sap['url1'] =df_sap['gold-url-1']
/var/folders/lp/ydkpkqws5mv6ptd1fy7p5w_40000gn/T/ipykernel_30661/1230006049.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sap['url2'] =df_sap['gold-url-2']
/var/folders/lp/ydkpkqws5mv6ptd1fy7p5w_40000gn/T/ipykernel_30661/1230006049.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [14]:
df_sap['score1'] =df_sap['gold-url-1']
df_sap['score2'] =df_sap['gold-url-2']
df_sap['score3'] =df_sap['gold-url-3']

/var/folders/lp/ydkpkqws5mv6ptd1fy7p5w_40000gn/T/ipykernel_30661/3898582753.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sap['score1'] =df_sap['gold-url-1']
/var/folders/lp/ydkpkqws5mv6ptd1fy7p5w_40000gn/T/ipykernel_30661/3898582753.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sap['score2'] =df_sap['gold-url-2']
/var/folders/lp/ydkpkqws5mv6ptd1fy7p5w_40000gn/T/ipykernel_30661/3898582753.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [15]:
df_sap.columns

Index(['question', 'Category', 'proposed-gold-url-1', 'proposed-gold-url-2',
       'proposed-gold-url-3', 'gold-url-1', 'gold-url-2', 'gold-url-3', 'url1',
       'url2', 'url3', 'score1', 'score2', 'score3'],
      dtype='object')

## Query Examples


### 1. Elser with Boost 

search_query1 = {
           "query":{
              "text_expansion":{
                 "ml.tokens":{
                    "model_id":".elser_model_1",
                    "model_text":"",
                     "boost":10
                 }
              }
           },
          "min_score": 10 ,
          "fields": [
            "filePath",
            "url"
          ],
          "_source": False
        }

In [ ]:
search_query1 = {
           "query":{
              "text_expansion":{
                 "ml.tokens":{
                    "model_id":".elser_model_1",
                    "model_text":"",
                     "boost":10
                 }
              }
           },
          "min_score": 10 ,
          "fields": [
            "filePath",
            "url"
          ],
          "_source": False
        }

### 2. Elser without BOOST 


In [ ]:

search_query1 = {
           "query":{
              "text_expansion":{
                 "ml.tokens":{
                    "model_id":".elser_model_1",
                    "model_text":""
                 }
              }
           },
          "min_score": 10 ,
          "fields": [
            "filePath",
            "url"
          ],
          "_source": False
        }

### 2. MULTIMATCH 


In [ ]:
search_query1 = {
  "query": {
   "multi_match" : {
      "query":]"",
      "type":"cross_fields",
      "analyzer":"standard", 
      "fields":[ "title", "text"]
    }
  },
  "min_score": 10 ,
  "fields": [
    "filePath",
    "url"
  ],
  "_source": False
  
}

### 3. MULTIMATCH with BOOST 


In [ ]:
search_query1 = {
  "query": {
   "multi_match" : {
      "query":"",
      "type":"cross_fields",
      "analyzer":"standard", 
      "fields":[ "title", "text"],
       "boost":10
    }
  },
  "min_score": 10 ,
  "fields": [
    "filePath",
    "url"
  ],
  "_source": False
  
}

### 4. without analyser


In [ ]:
search_query1 = {
  "query": {
   "multi_match" : {
      "query":"",
      "type":"cross_fields",
      "fields":[ "title", "text"],
       "boost":10
    }
  },
  "min_score": 10 ,
  "fields": [
    "filePath",
    "url"
  ],
  "_source": False
  
}

### 5. Best Field


In [ ]:
search_query1 = {
  "query": {
   "multi_match" : {
      "query":"",
      "type":"best_fields",
      "fields":[ "title", "text"],
       "tie_breaker": 0.3
       
    }
  },
  "min_score": 10 ,
  "fields": [
    "filePath",
    "url"
  ],
  "_source": False
  
}

### 6. bool query


In [ ]:
search_query1 = {
  "query": {
   "bool": {
      "must": {
        "match": {      
          "text": "How can I park supplier Invoices"
      }
    }
  
    }
  },
  "min_score": 10 ,
  "fields": [
    "filePath",
    "url"
  ],
  "_source": False
  
}

{
  "query": {
    
}
}

### 7. Elser with Cross field


In [ ]:
search_query1={
  "query": {
    "bool": { 
      "should": [
        {
          "text_expansion": {
            "ml.tokens": {
              "model_text": "Where do I find the APIs that are availbe in Sourcing and Procurment?",
              "model_id": ".elser_model_1"
            }
          }
        }
      ],
       "must": {
        "multi_match" : {
        "query":      "Where do I find the APIs that are availbe in Sourcing and Procurment?",
        "type":       "cross_fields",
        "analyzer":   "standard", 
        "fields":     [ "title", "text"]
    }
    }
    }
  },
  "min_score": 10 ,
  "fields": [
    "filePath",
    "url"
  ],
  "_source": False
}

### 8. Elser with best field


In [ ]:
search_query1={
          "query": {
            "bool": { 
              "should": [
                {
                  "text_expansion": {
                    "ml.tokens": {
                      "model_text":question,
                      "model_id": ".elser_model_1"
                    }
                  }
                }
              ],
               "must": {
                "multi_match" : {
                "query":question,
                "type":"best_fields",
                "fields":[ "title", "text"],
                "tie_breaker": 0.3
            }
            }
            }
              },
          "min_score": 10 ,
          "fields": [
            "filePath",
            "url"
          ],
          "_source": False
        }

In [ ]:
### 

search_query1 = {
  "query": {
   "multi_match" : {
      "query":"",
      "type":"phrase_prefix",
      "fields":[ "title", "text"]
       
    }
  },
  "min_score": 10 ,
  "fields": [
    "filePath",
    "url"
  ],
  "_source": False
  
}

 search_query1={
          "query": {
            "bool": { 
              "should": [
                {
                  "text_expansion": {
                    "ml.tokens": {
                      "model_text":question,
                      "model_id": ".elser_model_1"
                    }
                  }
                }
              ],
               "must": {
                "multi_match" : {
                "query":question,
                "type":"phrase_prefix",
                "fields":[ "title", "text"]
            }
            }
            }
              },
          "min_score": 10 ,
          "fields": [
            "filePath",
            "url"
          ],
          "_source": False
        }


### 9. bool_prefix with Elser


In [ ]:
search_query = {
          "query": {
            "bool": { 
              "should": [
                {
                  "text_expansion": {
                    "ml.tokens": {
                      "model_text":question,
                      "model_id": ".elser_model_1"
                    }
                  }
                }
              ],
               "must": {
                "multi_match" : {
                "query":question,
                "type":"bool_prefix",
                "fields":[ "title", "text"]
            }
            }
            }
              },
          "min_score": 10 ,
          "fields": [
            "filePath",
            "url"
          ],
          "_source": False
        }

### 10. Elser with boost fields


In [ ]:
{
          "query": {
            "bool": { 
              "should": [
                {
                  "text_expansion": {
                    "ml.tokens": {
                      "model_text":question,
                      "model_id": ".elser_model_1"
                    }
                  }
                }
              ],
               "must": {
                "multi_match" : {
                "query":question,
                "type":"most_fields",
                "fields":[ "title", "text"]
            }
            }
            }
              },
          "min_score": 10 ,
          "fields": [
            "filePath",
            "url"
          ],
          "_source": False
        }

{
          "query": {
            "bool": { 
              "should": [
                {
                  "text_expansion": {
                    "ml.tokens": {
                      "model_text":question,
                      "model_id": ".elser_model_1"
                    }
                  }
                }
              ],
               "must": {
                "multi_match" : {
                "query":question,
                "type":"most_fields",
                "fields":[ "title", "text"]
            }
            }
            }
              },
          "min_score": 10 ,
          "fields": [
            "filePath",
            "url"
          ],
          "_source": False
        }

## Question embedding
#### sentence-transformers__all-minilm-l6-v2


In [61]:
def processESIndex(df_sap):
    i =0
    for ind in df_sap.index:
        print("Processsing -----",i)
        question =df_sap['question'][ind]
        search_query1 = {
          "query": {
           "multi_match" : {
              "query":question,
              "type":"bool_prefix",
              "fields":[ "title", "text"]

            }
          },
          "min_score": 10 ,
          "fields": [
            "filePath",
            "url"
          ],
          "_source": False

        }
        response = es.search(
        index='elser-s4hana-business-39k-512t-256t-2023-07-26',
        body=search_query1,
        scroll='5m',  # Set the scroll timeout (e.g., 5 minutes)
        size=3  # Set the number of documents to retrieve per scroll
        )
        all_hits = response['hits']['hits']
        print(len(all_hits))
        j=0
        for num, doc in enumerate(all_hits):
            print ("DOC ID:", doc["_id"], "--->", doc, type(doc), "\n")
            print ("DOC Score:", doc["_score"], "\n")
            print ("DOC url :", doc["fields"]['url'], "\n")
            if j==0:
                df_sap['url1'][ind] =doc["fields"]['url']
                df_sap['score1'][ind] =doc["_score"]
            elif j==1:
                df_sap['url2'][ind] =doc["fields"]['url']
                df_sap['score2'][ind] =doc["_score"]
            elif j==2:
                df_sap['url3'][ind] =doc["fields"]['url']
                df_sap['score3'][ind] =doc["_score"]
            j=j+1
        i =i+1
    return df_sap
        

In [ ]:
df_sap =processESIndex(df_sap)

In [63]:
df_sap.columns

Index(['question', 'Category', 'proposed-gold-url-1', 'proposed-gold-url-2',
       'proposed-gold-url-3', 'gold-url-1', 'gold-url-2', 'gold-url-3', 'url1',
       'url2', 'url3', 'score1', 'score2', 'score3'],
      dtype='object')

In [64]:
df_sap.head()

,question,Category,proposed-gold-url-1,proposed-gold-url-2,proposed-gold-url-3,gold-url-1,gold-url-2,gold-url-3,url1,url2,url3,score1,score2,score3
0,How do I get ready for using Sourcing and Proc...,Indirect,https://help.sap.com/docs/SAP_S4HANA_CLOUD/bb9...,NaN,NaN,https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e6...,NaN,NaN,[https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e...,[https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e...,[https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e...,45.986645,44.425552,40.902496
1,What features does SAP Sourcing and Procuremen...,Direct,https://help.sap.com/docs/SAP_S4HANA_CLOUD/bb9...,NaN,NaN,https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e6...,NaN,NaN,[https://help.sap.com/docs/SAP_S4HANA_CLOUD/63...,[https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e...,[https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e...,28.099304,27.13894,26.921707
2,What integration scenarios are available for S...,Direct,https://help.sap.com/docs/SAP_S4HANA_CLOUD/bb9...,NaN,NaN,https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e6...,NaN,NaN,[https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e...,[https://help.sap.com/docs/SAP_S4HANA_CLOUD/b2...,[https://help.sap.com/docs/SAP_S4HANA_CLOUD/b2...,50.218422,48.836723,48.836723
3,Can I connect my SAP S/4HANA Cloud with extern...,Indirect,https://help.sap.com/docs/SAP_S4HANA_CLOUD/bb9...,NaN,NaN,https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e6...,NaN,NaN,[https://help.sap.com/docs/SAP_S4HANA_CLOUD/6b...,[https://help.sap.com/docs/SAP_S4HANA_CLOUD/6b...,[https://help.sap.com/docs/SAP_S4HANA_CLOUD/6b...,53.618404,50.663227,49.49715
4,What does Central Procurement do?,Direct,https://help.sap.com/docs/SAP_S4HANA_CLOUD/bb9...,NaN,NaN,https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e6...,NaN,NaN,[https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e...,[https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e...,[https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e...,25.9769,22.494854,22.246145


In [65]:
df_sap.to_excel('SAPBool_prefixwithoutBoost.xlsx')

### Execute Queries over the entire dataset

In [109]:
## Run the query :
for ind in df_sap.index:
    #print(ind)
    #print(df_sap['question'][ind],df_sap['Category'][ind])
    question =df_sap['question'][ind]
    cat = df_sap['Category'][ind]

In [119]:
search_query1 = {
   "query":{
      "text_expansion":{
         "ml.tokens":{
            "model_id":".elser_model_1",
            "model_text":"Where do I work on purchase contracts contracts?",
            "boost":10
         }
      }
   },
  "min_score": 10 ,
  "fields": [
    "filePath",
    "url"
  ],
  "_source": False
}

In [120]:
# Perform the initial search to obtain the first batch of results
response = es.search(
    index='elser-s4hana-business-39k-512t-256t-2023-07-26',
    body=search_query1,
    scroll='5m',  # Set the scroll timeout (e.g., 5 minutes)
    size=3  # Set the number of documents to retrieve per scroll
)
all_hits = response['hits']['hits']
print(len(all_hits))

10


In [124]:
# iterate the nested dictionaries inside the ["hits"]["hits"] list
for num, doc in enumerate(all_hits):
    print ("DOC ID:", doc["_id"], "--->", doc, type(doc), "\n")
    print ("DOC Score:", doc["_score"], "\n")
    print ("DOC url :", doc["fields"]['url'], "\n")
    
    #print ("DOC URL:", doc["_source"]["url"], "--->", "\n")
    #print ("DOC Source:", doc["_source"]["published_source"], "\n")
    #print ("keywords:", doc["_source"]["keywords"], "\n")
    #print ("DOC content:", doc["_source"]["content"], "\n")
    

    # print a few spaces between each doc for readability
    print ("\n\n")

DOC ID: hana_05638.txt-0-17 ---> {'_index': 'elser-s4hana-business-39k-512t-256t-2023-07-26', '_id': 'hana_05638.txt-0-17', '_score': 158.58463, 'fields': {'url': ['https://help.sap.com/docs/SAP_S4HANA_CLOUD/bb9f1469daf04bd894ab2167f8132a1a/400b07d741274f4893372189181d1480.html'], 'filePath': ['400b07d741274f4893372189181d1480.html']}} <class 'dict'> 

DOC Score: 158.58463 

DOC url : ['https://help.sap.com/docs/SAP_S4HANA_CLOUD/bb9f1469daf04bd894ab2167f8132a1a/400b07d741274f4893372189181d1480.html'] 




DOC ID: hana_29224.txt-16429-18681 ---> {'_index': 'elser-s4hana-business-39k-512t-256t-2023-07-26', '_id': 'hana_29224.txt-16429-18681', '_score': 151.82756, 'fields': {'url': ['https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e602d466b99490187fcbb30d1dc897c/a8bbfa839c394505a7557f8171637d1a.html'], 'filePath': ['a8bbfa839c394505a7557f8171637d1a.html']}} <class 'dict'> 

DOC Score: 151.82756 

DOC url : ['https://help.sap.com/docs/SAP_S4HANA_CLOUD/0e602d466b99490187fcbb30d1dc897c/a8bbfa839